# Sentence - CNNs

In [ ]:
# Mount drive and read input files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls
%cd 'drive/MyDrive/266 Final Project/'
!ls

drive  sample_data
/content/drive/.shortcut-targets-by-id/1ab9FCOKtHUQO8mIh4jjbS-JZlvWC8W54/266 Final Project
'11 10 Working Session.gdoc'			      'Model Experimentation - Sentence Highlights'
 baseline.ipynb					      'Prediction Data'
 Experimentation.gsheet				      'Processed Data'
'FINAL SUBMISSION'				       Resources
'Full Orchestration: Stage 1+2'			       simple_baseline.ipynb
'Input Data'					      'Stage 1'
 match_analysis.ipynb				      'Stage 2'
 model_checkpointst5base-finetuned-seqclassification  'Validation Set.xlsx'
'Model Experimentation - Chunk Highlights'


In [ ]:
!pip install gensim --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text --quiet
!pip install pydot --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 89.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import pandas as pd
import numpy as np
import tensorflow as tf
import keras

from keras.layers import Embedding, Input, Dense, Lambda
from keras.models import Model
#import keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re


import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

In [ ]:
# 4-window plot of loss and accuracy for two models, for comparison

def make_plot(axs,
              model_history1,
              model_history2,
              model_1_name='model 1',
              model_2_name='model 2',
              ):
    box = dict(facecolor='yellow', pad=5, alpha=0.2)

    for i, metric in enumerate(['loss', 'accuracy']):
        y_lim_lower1 = np.min(model_history1.history[metric])
        y_lim_lower2 = np.min(model_history2.history[metric])
        y_lim_lower = min(y_lim_lower1, y_lim_lower2) * 0.9

        y_lim_upper1 = np.max(model_history1.history[metric])
        y_lim_upper2 = np.max(model_history2.history[metric])
        y_lim_upper = max(y_lim_upper1, y_lim_upper2) * 1.1

        for j, model_history in enumerate([model_history1, model_history2]):
            model_name = [model_1_name, model_2_name][j]
            ax1 = axs[i, j]
            ax1.plot(model_history.history[metric])
            ax1.plot(model_history.history['val_%s' % metric])
            ax1.set_title('%s - %s' % (metric, model_name))
            ax1.set_ylabel(metric, bbox=box)
            ax1.set_ylim(y_lim_lower, y_lim_upper)

#Read Data

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/266 Final Project/Full Orchestration: Stage 1+2/final_data/Old Data/sentence_training_data.csv')
test_data = pd.read_csv('/content/drive/MyDrive/266 Final Project/Full Orchestration: Stage 1+2/final_data/Old Data/sentence_testing_data.csv')

In [ ]:
train_data.head()

,service_id,doc_text,highlight,label
0,614,Cookies are widely used because they allow a w...,NaN,0.0
1,403,You release us from liability concerning 5.2.1...,NaN,0.0
2,186,"Permissions section of your Account Settings, ...",NaN,0.0
3,591,"You acknowledge that you have only a limited, ...",NaN,0.0
4,1209,11.0 CONFIDENTIAL INFORMATION\n\n11.1 Confiden...,NaN,0.0


In [ ]:
train_data['label'].value_counts()

,count
label,
0.0,2993
1.0,2993


In [ ]:
test_data['label'].value_counts()

,count
label,
0.0,8872
1.0,748


In [ ]:
X_train = train_data['doc_text']
X_test = test_data['doc_text']
y_train = train_data['label']
y_test = test_data['label']

In [ ]:
tokenizer = tf_text.WhitespaceTokenizer()
train_tokens = tokenizer.tokenize(X_train)
test_tokens = tokenizer.tokenize(X_test)

In [ ]:
train_tokens[0]

<tf.Tensor: shape=(19,), dtype=string, numpy=
array([b'Cookies', b'are', b'widely', b'used', b'because', b'they',
       b'allow', b'a', b'website', b'to', b'recognise', b'a', b'users',
       b'device', b'and', b'therefore', b'allow', b'enhanced',
       b'functionality.'], dtype=object)>

In [ ]:
nltk.download('word2vec_sample')

word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))

wvmodel = KeyedVectors.load_word2vec_format(datapath(word2vec_sample), binary=False)

[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Package word2vec_sample is already up-to-date!


In [ ]:
EMBEDDING_DIM = 300

# initialize embedding matrix and word-to-id map:
embedding_matrix = np.zeros((len(wvmodel) + 1, EMBEDDING_DIM))
vocab_dict = {}

# build the embedding matrix and the word-to-id map:
for i, word in enumerate(wvmodel.index_to_key):
    embedding_vector = wvmodel[word]

    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        vocab_dict[word] = i

# we can use the last index at the end of the vocab for unknown tokens
vocab_dict['[UNK]'] = len(vocab_dict)

In [ ]:
MAX_SEQUENCE_LENGTH = 256

In [ ]:
def docs_to_vocab_ids(tokenized_texts_list, MAX_SEQUENCE_LENGTH):
    """
    converting a list of strings to a list of lists of word ids
    """
    texts_vocab_ids = []
    text_labels = []
    valid_example_list = []
    for i, token_list in enumerate(tokenized_texts_list):

        # Get the vocab id for each token in this doc ([UNK] if not in vocab)
        vocab_ids = []
        for token in list(token_list.numpy()):
            decoded = token.decode('utf-8', errors='ignore')
            if decoded in vocab_dict:
                vocab_ids.append(vocab_dict[decoded])
            else:
                vocab_ids.append(vocab_dict['[UNK]'])

        # Truncate text to max length, add padding up to max length
        vocab_ids = vocab_ids[:MAX_SEQUENCE_LENGTH]
        n_padding = (MAX_SEQUENCE_LENGTH - len(vocab_ids))
        # For simplicity in this model, we'll just pad with unknown tokens
        vocab_ids += [vocab_dict['[UNK]']] * n_padding
        # Add this example to the list of converted docs
        texts_vocab_ids.append(vocab_ids)

        if i % 5000 == 0:
            print('Examples processed: ', i)

    print('Total examples: ', i)
    return np.array(texts_vocab_ids)

In [ ]:
train_input = docs_to_vocab_ids(train_tokens, MAX_SEQUENCE_LENGTH)
test_input = docs_to_vocab_ids(test_tokens, MAX_SEQUENCE_LENGTH)

train_labels = np.array(y_train)
test_labels = np.array(y_test)

Examples processed:  0
Examples processed:  5000
Total examples:  5985
Examples processed:  0
Examples processed:  5000
Total examples:  9619


In [ ]:
def build_cnn_model(num_filters, kernel_sizes, dense_layer_dims, dropout_rate, embeddings_trainable=False):
    cnn_input_layer = keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int64')
    cnn_embedding_layer = Embedding(embedding_matrix.shape[0],
                                    embedding_matrix.shape[1],
                                    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                                    input_length=MAX_SEQUENCE_LENGTH,
                                    trainable=embeddings_trainable)
    cnn_embeddings = cnn_embedding_layer(cnn_input_layer)

    conv_layers_for_all_kernel_sizes = []
    for kernel_size, filters in zip(kernel_sizes, num_filters):
        conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(cnn_embeddings)
        conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
        conv_layers_for_all_kernel_sizes.append(conv_layer)

    conv_output = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1)
    last_hidden_output = keras.layers.Dropout(rate=dropout_rate)(conv_output)

    for dense_layer_dim in dense_layer_dims:
        last_hidden_output = keras.layers.Dense(dense_layer_dim, activation='relu')(last_hidden_output)

    cnn_prediction = keras.layers.Dense(1, activation='sigmoid')(last_hidden_output)

    cnn_model = keras.Model(inputs=cnn_input_layer, outputs=cnn_prediction)
    cnn_model.compile(optimizer='adam',
                      loss='binary_crossentropy',  # From information theory notebooks.
                      metrics=['accuracy'])
    return cnn_model

In [ ]:
# Now specify model hyperparameters and call the function to create a CNN model

num_filters = [100, 100, 50, 25]
kernel_sizes = [3, 4, 5, 10]
dense_layer_dims = [100, 50]
dropout_rate = 0.5

cnn_model = build_cnn_model(num_filters, kernel_sizes, dense_layer_dims, dropout_rate)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
cnn_history = cnn_model.fit(train_input,
                            y_train,
                            validation_data=(test_input, y_test),
                            batch_size=32,
                            epochs=5
                            )

Epoch 1/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 123s 634ms/step - accuracy: 0.5195 - loss: 0.6906 - val_accuracy: 0.5666 - val_loss: 0.6708
Epoch 2/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 139s 622ms/step - accuracy: 0.6084 - loss: 0.6574 - val_accuracy: 0.3260 - val_loss: 0.8121
Epoch 3/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 141s 619ms/step - accuracy: 0.6539 - loss: 0.6170 - val_accuracy: 0.5898 - val_loss: 0.6444
Epoch 4/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 115s 613ms/step - accuracy: 0.7210 - loss: 0.5490 - val_accuracy: 0.4858 - val_loss: 0.7570
Epoch 5/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 157s 840ms/step - accuracy: 0.7678 - loss: 0.4963 - val_accuracy: 0.6413 - val_loss: 0.6291


In [ ]:
predictions = cnn_model.predict(test_input)


301/301 ━━━━━━━━━━━━━━━━━━━━ 47s 157ms/step
